In [10]:
with open('test.txt') as metabolites_file:
    n_tests = int(metabolites_file.readline().strip())
    print(f'n_tests: {n_tests}')
    for _ in range(n_tests):
        int_numbers = metabolites_file.readline().strip()
        nM, nK, nN = [int(x) for x in int_numbers.split()]
        M = metabolites_file.readline().strip()
        N = metabolites_file.readline().strip()
        K = metabolites_file.readline().strip()
        print(f'nM: {nM}, nK: {nK}, nN: {nN}')

n_tests: 3
nM: 2, nK: 2, nN: 5
nM: 2, nK: 2, nN: 5
nM: 5, nK: 4, nN: 7


In [16]:
def process_test(M, N, K):
    pass

with open('test.txt') as metabolites_file:
    n_tests = int(metabolites_file.readline().strip())
    print(f'n_tests: {n_tests}')
    for _ in range(n_tests):
        int_numbers = metabolites_file.readline().strip()
        nM, nK, nN = [int(x) for x in int_numbers.split()]
        print(f'nM: {nM}, nK: {nK}, nN: {nN}')
        M = [float(x) for x in metabolites_file.readline().strip().split()]
        K = [float(x) for x in metabolites_file.readline().strip().split()]
        N = [float(x) for x in metabolites_file.readline().strip().split()]
        print(f'M: {M}')
        print(f'K: {K}')
        print(f'N: {N}')
        # iM, iK = process_test(M, N, K)

n_tests: 3
nM: 2, nK: 2, nN: 5
M: [1.000002, 2e-06]
K: [0.5, -0.5]
N: [0.500001, 0.500002, 0.500003, 1.0, 1e-06]
nM: 2, nK: 2, nN: 5
M: [1.000002, 1e-06]
K: [0.5, -0.5]
N: [0.500001, 0.500002, 0.500003, 1.0, 1e-06]
nM: 5, nK: 4, nN: 7
M: [1e-06, 2e-06, 3e-06, 4e-06, 5e-06]
K: [2e-06, 1e-05, 1e-06, -1e-06]
N: [1e-06, 2e-06, 0.0001, 5e-06, 2e-05, 1e-05, 3e-06]


In [10]:
def process_test(M, N, K):
    nM = len(M)
    nN = len(N)
    nK = len(K)
    i_out = []
    delta = 0
    min_delta = 0
    m_k_sum = 0
    for i in range(len(N)):
        iM_iK_list = []
        for j in range(len(M)):
            for k in range(len(K)):
                m_k_sum = M[j] + K[k]
                if m_k_sum > 0 and m_k_sum != N[i]:
                    delta = abs(m_k_sum - N[i])
                if delta < min_delta:
                    min_delta = delta
                if m_k_sum == N[i] or min_delta == delta:
                    iM_iK_list.append(j)
                    iM_iK_list.append(k)
                    i_out.append(iM_iK_list)
        print(f'N[i]: {N[i]}')
        print(f'M[j]: {M[j]}')
        print(f'K[k]: {K[k]}')
        print(f'i_out: {i_out}')
    return i_out

def annotate(input_name):
    test_file = open(input_name)
    n_tests = int(test_file.readline().strip())
    print(f'n_tests: {n_tests}')
    for _ in range(n_tests):
        int_numbers = test_file.readline().strip()
        nM, nK, nN = [int(x) for x in int_numbers.split()]
        print(f'nM: {nM}, nK: {nK}, nN: {nN}')
        M = [float(x) for x in test_file.readline().strip().split()]
        K = [float(x) for x in test_file.readline().strip().split()]
        N = [float(x) for x in test_file.readline().strip().split()]
        print(f'M: {M}')
        print(f'K: {K}')
        print(f'N: {N}')
        iM, iK = process_test(M, N, K)
    return iM, iK

input_name = 'test.txt'
annotate(input_name)

n_tests: 3
nM: 2, nK: 2, nN: 5
M: [1.000002, 2e-06]
K: [0.5, -0.5]
N: [0.500001, 0.500002, 0.500003, 1.0, 1e-06]
N[i]: 0.500001
M[j]: 2e-06
K[k]: -0.5
i_out: []
N[i]: 0.500002
M[j]: 2e-06
K[k]: -0.5
i_out: [[1, 0]]
N[i]: 0.500003
M[j]: 2e-06
K[k]: -0.5
i_out: [[1, 0]]
N[i]: 1.0
M[j]: 2e-06
K[k]: -0.5
i_out: [[1, 0]]
N[i]: 1e-06
M[j]: 2e-06
K[k]: -0.5
i_out: [[1, 0]]


ValueError: not enough values to unpack (expected 2, got 1)

In [23]:
def process_test(M, K, N):
    """
    M is database of metabolites
    K is database of adducts
    N is our measured signals
    """
    list_pairs = []
    for isignal in range(len(N)):
        meta_min = 0
        adduct_min = 0
        delta_min = abs(M[meta_min] + K[adduct_min] - N[isignal])
        for imeta in range(len(M)):
            for iadduct in range(len(K)):
                delta_curr = abs(M[imeta] + K[iadduct] - N[isignal])
                if delta_curr < delta_min:
                    delta_min = delta_curr
                    meta_min = imeta
                    adduct_min = iadduct

        # here we selected pair that gives closest sum to our signal
        list_pairs.append((meta_min, adduct_min))

    return list_pairs 

def annotate(input_name, output_name):
    input_file = open(input_name)
    output_file = open(output_name, 'w')

    n_tests = int(input_file.readline().strip())
    for itest in range(n_tests):
        int_numbers = input_file.readline().strip()
        nM, nK, nN = [int(x) for x in int_numbers.split()]
        print(f'nM: {nM}, nK: {nK}, nN: {nN}')
        M = [float(x) for x in input_file.readline().strip().split()]
        K = [float(x) for x in input_file.readline().strip().split()]
        N = [float(x) for x in input_file.readline().strip().split()]
        
        # print(f'\ntest {itest}')
        # print(f'M: {M}')
        # print(f'K: {K}')
        # print(f'N: {N}')

        list_pairs = process_test(M, K, N)
        
        for x in list_pairs:
            output_file.write(f'{x[0] + 1} {x[1] + 1}\n')
            print(f'{x[0] + 1} {x[1] + 1}')

    print(f'created output file {output_name}')
    input_file.close()
    output_file.close()

input_name = 'test.txt'
output_name = 'test_output.txt'
annotate(input_name, output_name)

nM: 2, nK: 2, nN: 5
2 1
2 1
1 2
1 2
2 2
nM: 2, nK: 2, nN: 5
2 1
1 2
1 2
1 2
2 1
nM: 5, nK: 4, nN: 7
2 4
1 3
5 2
3 1
5 2
1 2
1 1
created output file test_output.txt


In [3]:
def process_test(M, K, N):
    """
    M is database of metabolites
    K is database of adducts
    N is our measured signals
    """
    list_pairs = []
    for isignal in range(len(N)):
        meta_min = None
        adduct_min = None
        # delta_min = abs(M[meta_min] + K[adduct_min] - N[isignal])
        delta_min = 1e6
        for imeta in range(len(M)):
            for iadduct in range(len(K)):
                sum_curr = M[imeta] + K[iadduct]
                if sum_curr < 0:
                    continue
                delta_curr = abs(sum_curr - N[isignal])
                if delta_curr < delta_min:
                    delta_min = delta_curr
                    meta_min = imeta
                    adduct_min = iadduct

        # here we selected pair that gives closest sum to our signal
        if meta_min is None:
            print(f'isignal: {isignal} meta_min is None')
        if adduct_min is None:
            print(f'isignal: {isignal} adduct_min is None')
        list_pairs.append((meta_min, adduct_min))

    return list_pairs 

def annotate(input_name, output_name):
    input_file = open(input_name)
    output_file = open(output_name, 'w')

    n_tests = int(input_file.readline().strip())
    for itest in range(n_tests):
        print(f'itest: {itest}')

        int_numbers = input_file.readline().strip()
        nM, nK, nN = [int(x) for x in int_numbers.split()]
        # print(f'nM: {nM}, nK: {nK}, nN: {nN}')

        M = [float(x) for x in input_file.readline().strip().split()]
        K = [float(x) for x in input_file.readline().strip().split()]
        N = [float(x) for x in input_file.readline().strip().split()]
        
        # print(f'M: {M}')
        # print(f'K: {K}')
        # print(f'N: {N}')

        list_pairs = process_test(M, K, N)
        
        for x in list_pairs:
            print(f'{x[0] + 1} {x[1] + 1}')
            output_file.write(f'{x[0] + 1} {x[1] + 1}\n')

    print(f'created output file {output_name}')
    input_file.close()
    output_file.close()

input_name = 'test.txt'
output_name = 'test_output.txt'
annotate(input_name, output_name)

itest: 0
2 1
2 1
1 2
1 2
2 1
itest: 1
2 1
1 2
1 2
1 2
2 1
itest: 2
2 4
1 3
5 2
3 1
5 2
1 2
1 1
created output file test_output.txt


In [2]:
%%time

# for test #1

def process_test(M, K, N):
    """
    M is database of metabolites
    K is database of adducts
    N is our measured signals
    """
    list_pairs = []
    for isignal in range(len(N)):
        meta_min = None
        adduct_min = None
        # delta_min = abs(M[meta_min] + K[adduct_min] - N[isignal])
        delta_min = 1e6
        for imeta in range(len(M)):
            for iadduct in range(len(K)):
                sum_curr = M[imeta] + K[iadduct]
                if sum_curr < 0:
                    continue
                delta_curr = abs(sum_curr - N[isignal])
                if delta_curr < delta_min:
                    delta_min = delta_curr
                    meta_min = imeta
                    adduct_min = iadduct

        # here we selected pair that gives closest sum to our signal
        if meta_min is None:
            print(f'isignal: {isignal} meta_min is None')
        if adduct_min is None:
            print(f'isignal: {isignal} adduct_min is None')
        list_pairs.append((meta_min, adduct_min))

    return list_pairs 

def annotate(input_name, output_name):
    input_file = open(input_name)
    output_file = open(output_name, 'w')

    n_tests = int(input_file.readline().strip())
    for itest in range(n_tests):

        int_numbers = input_file.readline().strip()
        nM, nK, nN = [int(x) for x in int_numbers.split()]

        M = [float(x) for x in input_file.readline().strip().split()]
        K = [float(x) for x in input_file.readline().strip().split()]
        N = [float(x) for x in input_file.readline().strip().split()]
        
        list_pairs = process_test(M, K, N)
        
        for x in list_pairs:
            output_file.write(f'{x[0] + 1} {x[1] + 1}\n')

    print(f'created output file {output_name}')
    input_file.close()
    output_file.close()

input_name = '1.txt'
output_name = '1_output.txt'
annotate(input_name, output_name)

created output file 1_output.txt
Wall time: 3 ms


In [4]:
%%time

# for test #2

def process_test(M, K, N):
    """
    M is database of metabolites
    K is database of adducts
    N is our measured signals
    """
    list_pairs = []
    for isignal in range(len(N)):
        meta_min = None
        adduct_min = None
        # delta_min = abs(M[meta_min] + K[adduct_min] - N[isignal])
        delta_min = 1e6
        for imeta in range(len(M)):
            for iadduct in range(len(K)):
                sum_curr = M[imeta] + K[iadduct]
                if sum_curr < 0:
                    continue
                delta_curr = abs(sum_curr - N[isignal])
                if delta_curr < delta_min:
                    delta_min = delta_curr
                    meta_min = imeta
                    adduct_min = iadduct

        # here we selected pair that gives closest sum to our signal
        if meta_min is None:
            print(f'isignal: {isignal} meta_min is None')
        if adduct_min is None:
            print(f'isignal: {isignal} adduct_min is None')
        list_pairs.append((meta_min, adduct_min))

    return list_pairs 

def annotate(input_name, output_name):
    input_file = open(input_name)
    output_file = open(output_name, 'w')

    n_tests = int(input_file.readline().strip())
    for itest in range(n_tests):

        int_numbers = input_file.readline().strip()
        nM, nK, nN = [int(x) for x in int_numbers.split()]

        M = [float(x) for x in input_file.readline().strip().split()]
        K = [float(x) for x in input_file.readline().strip().split()]
        N = [float(x) for x in input_file.readline().strip().split()]
        
        list_pairs = process_test(M, K, N)
        
        for x in list_pairs:
            output_file.write(f'{x[0] + 1} {x[1] + 1}\n')

    print(f'created output file {output_name}')
    input_file.close()
    output_file.close()

input_name = '2.txt'
output_name = '2_output.txt'
annotate(input_name, output_name)

created output file 2_output.txt
Wall time: 2min 34s


In [ ]:
%%time

# for test #4

def process_test(M, K, N):
    """
    M is database of metabolites
    K is database of adducts
    N is our measured signals
    """
    list_pairs = []
    for isignal in range(len(N)):
        meta_min = None
        adduct_min = None
        # delta_min = abs(M[meta_min] + K[adduct_min] - N[isignal])
        delta_min = 1e6
        for imeta in range(len(M)):
            for iadduct in range(len(K)):
                sum_curr = M[imeta] + K[iadduct]
                if sum_curr < 0:
                    continue
                delta_curr = abs(sum_curr - N[isignal])
                if delta_curr < delta_min:
                    delta_min = delta_curr
                    meta_min = imeta
                    adduct_min = iadduct

        # here we selected pair that gives closest sum to our signal
#         if meta_min is None:
#             print(f'isignal: {isignal} meta_min is None')
#         if adduct_min is None:
#             print(f'isignal: {isignal} adduct_min is None')
        list_pairs.append((meta_min, adduct_min))

    return list_pairs 

def annotate(input_name, output_name):
    input_file = open(input_name)
    output_file = open(output_name, 'w')

    n_tests = int(input_file.readline().strip())
    for itest in range(n_tests):

        int_numbers = input_file.readline().strip()
        nM, nK, nN = [int(x) for x in int_numbers.split()]

        M = [float(x) for x in input_file.readline().strip().split()]
        K = [float(x) for x in input_file.readline().strip().split()]
        N = [float(x) for x in input_file.readline().strip().split()]
        
        list_pairs = process_test(M, K, N)
        
        for x in list_pairs:
            output_file.write(f'{x[0] + 1} {x[1] + 1}\n')

    print(f'created output file {output_name}')
    input_file.close()
    output_file.close()

input_name = '4.txt'
output_name = '4_output.txt'
annotate(input_name, output_name)

In [13]:
list_pairs = [(10, 11), (20, 21), (30, 31)]
list_pairs_new = [(x[0] + 100, x[1] + 100) for x in list_pairs]
print(list_pairs_new)

[(110, 111), (120, 121), (130, 131)]


In [3]:
orig = [20, 10, 15, 30, 25]
ind = [1, 2, 0, 4, 3]

for i in range(len(orig)):
    print(f'{orig[i]}')
print()

for i in range(len(orig)):
    print(f'{orig[ind[i]]}')

20
10
15
30
25

10
15
20
25
30


In [7]:
import numpy as np

orig = [20, 10, 15, 30, 25]
# ind = [1, 2, 0, 4, 3]

print(f'orig: {orig}')
print()

ind = np.argsort(orig)

for i in range(len(orig)):
    print(f'{orig[ind[i]]}')

print(f'orig: {orig}')

orig: [20, 10, 15, 30, 25]

10
15
20
25
30
orig: [20, 10, 15, 30, 25]


In [16]:
M = [0.000003, 0.000012, 0.000081, 0.000099, 0.000076, 0.000045, 0.000092, 0.000068, 0.000047]
K = [0.000002, 0.000045, -0.000063, -0.000009, -0.000050, 0.000048, 0.000070, -0.000037, 0.000056, -0.000008] 
N = [0.000079, 0.000094, 0.000084, 0.000052, 0.000064, 0.000055, 0.000070, 0.000079]

k_indices = np.argsort(K)  # returns indices
print(f'k_indices: {k_indices}')

k_sorted = []
for i in range(len(K)):
    k_sorted.append(K[k_indices[i]])
print(f'k_sorted: {k_sorted}')

print()
m_indices = np.argsort(M)
print(f'm_indices: {m_indices}')

m_sorted = []
for i in range(len(M)):
    m_sorted.append(M[m_indices[i]])
print(f'm_sorted: {m_sorted}')

k_indices: [2 4 7 3 9 0 1 5 8 6]
k_sorted: [-6.3e-05, -5e-05, -3.7e-05, -9e-06, -8e-06, 2e-06, 4.5e-05, 4.8e-05, 5.6e-05, 7e-05]

m_indices: [0 1 5 8 7 4 2 6 3]
m_sorted: [3e-06, 1.2e-05, 4.5e-05, 4.7e-05, 6.8e-05, 7.6e-05, 8.1e-05, 9.2e-05, 9.9e-05]


In [13]:
def binary_search(array, low, high, x):
    if high >= low:
        midpt = (high + low) // 2
        
        if array[midpt] == x:
            # found: stop recursion
            return midpt

        # continue search in the half that contains element
        
        if array[midpt] > x:
            return binary_search(array, low, midpt - 1, x)
        else:
            return binary_search(array, midpt + 1, high, x)

    else:
        return -1

# array = [-6.3e-05, -5e-05, -3.7e-05, -9e-06, -8e-06, 2e-06, 4.5e-05, 4.8e-05, 5.6e-05, 7e-05]
# x = 4.5e-5
array = [ 2, 3, 4, 10, 40, 50]
x = 10
x = 2
x = 50
x = 10
index = binary_search(array, 0, len(array) -1, x)
print(f'found {x} at index {index}')

found 10 at index 3


In [20]:
def binary_search(array, low, high, x):
    if high >= low:
        midpt = (high + low) // 2
        
        # eps = 1e-12
        eps = 1e-3
        if array[midpt] == x:
        # if abs(array[midpt] - x) < eps:
            # found: stop recursion
            return midpt

        # continue search in the half that contains element
        
        if array[midpt] > x:
            return binary_search(array, low, midpt - 1, x)
        else:
            return binary_search(array, midpt + 1, high, x)

    else:
        return -1

# array = [-6.3e-05, -5e-05, -3.7e-05, -9e-06, -8e-06, 2e-06, 4.5e-05, 4.8e-05, 5.6e-05, 7e-05]
# x = 4.5e-5
array = [ 2, 3, 1.333333, 10, 40, 50]
x = 10
x = 2
x = 50
x = 4 / 3
index = binary_search(array, 0, len(array) -1, x)
print(f'found {x} at index {index}')

found 1.3333333333333333 at index -1


In [ ]:
def binary_search(array, low, high, x):
    if high >= low:
        midpt = (high + low) // 2
        
        eps = 1e-7
        if array[midpt] == x:
            # found: stop recursion
            return midpt

        # continue search in the half that contains element
        
        if array[midpt] > x:
            return binary_search(array, low, midpt - 1, x)
        else:
            return binary_search(array, midpt + 1, high, x)

    else:
        return -1

def process_test(M, K, N):
    """
    M is database of metabolites
    K is database of adducts
    N is our measured signals
    """
    list_pairs = []
    for isignal in range(len(N)):
        meta_min = None
        adduct_min = None
        # delta_min = abs(M[meta_min] + K[adduct_min] - N[isignal])
        delta_min = 1e6
        for imeta in range(len(M)):
            for iadduct in range(len(K)):
                sum_curr = M[imeta] + K[iadduct]
                if sum_curr < 0:
                    continue
                delta_curr = abs(sum_curr - N[isignal])
                if delta_curr < delta_min:
                    delta_min = delta_curr
                    meta_min = imeta
                    adduct_min = iadduct

        # here we selected pair that gives closest sum to our signal
        if meta_min is None:
            print(f'isignal: {isignal} meta_min is None')
        if adduct_min is None:
            print(f'isignal: {isignal} adduct_min is None')
        list_pairs.append((meta_min, adduct_min))

    return list_pairs


def process_test_sorted(M, K_sorted, k_indices, N):
    """
    M is database of metabolites
    K_sorted is database of adducts sorted
    k_indices is list of indices returned by np.argsort
    N is our measured signals
    """
    list_pairs = []
    adduct_dict = {}  # key: meta, value: adduct that when added to meta gives signal
    k_ind_dict = {}  # key: sorted indices, value: orig indices
    
    # find imeta in M, iadduct in sorted array
    for isignal in range(len(N)):
        for imeta in range(len(M)):
            diff_curr = N[isignal] - M[imeta]
            if diff_curr not in adduct_dict:
                adduct_dict[M[imeta]] = diff_curr

            adduct_index = binary_search(K_sorted, 0, len(K_sorted) -1, diff_curr)  # indices in sorted array
    
    list_pairs.append(imeta, adduct_dict[iadduct])
    return imeta, iadduct

In [13]:
def binary_search(array, low, high, x):
    if high >= low:
        midpt = (high + low) // 2
        
        if array[midpt] == x:
            # found: stop recursion
            return midpt

        # continue search in the half that contains element
        
        if array[midpt] > x:
            return binary_search(array, low, midpt - 1, x)
        else:
            return binary_search(array, midpt + 1, high, x)

    else:
        return -1

# array = [-6.3e-05, -5e-05, -3.7e-05, -9e-06, -8e-06, 2e-06, 4.5e-05, 4.8e-05, 5.6e-05, 7e-05]
# x = 4.5e-5
array = [ 2, 3, 4, 10, 40, 50]
x = 10
x = 2
x = 50
x = 10
index = binary_search(array, 0, len(array) -1, x)
print(f'found {x} at index {index}')

found 10 at index 3


In [1]:
import numpy as np

orig = [20, 10, 15, 30, 25]
# ind = [1, 2, 0, 4, 3]

print(f'orig: {orig}')
print()

ind = np.argsort(orig)
print(f'indices: {ind}')

for i in range(len(orig)):
    print(f'{orig[ind[i]]}')

print(f'orig: {orig}')

orig: [20, 10, 15, 30, 25]

indices: [1 2 0 4 3]
10
15
20
25
30
orig: [20, 10, 15, 30, 25]


In [1]:
import numpy as np

orig = [20, 10, 15, 30, 25]
# ind = [1, 2, 0, 4, 3]

print(f'orig: {orig}')
print()

ind = np.argsort(orig)
print(f'indices: {ind}')

for i in range(len(orig)):
    print(f'{orig[ind[i]]}')

print(f'orig: {orig}')

orig: [20, 10, 15, 30, 25]

indices: [1 2 0 4 3]
10
15
20
25
30
orig: [20, 10, 15, 30, 25]


# Modify Binary Search
Use two arrays: *original* array and _sorted_ indices.  
Toy example.

In [1]:
import numpy as np

orig = [20, 10, 15, 30, 25]
sort_indices = np.argsort(orig)
print(f'orig: {orig}')
print(f'sort_indices: {sort_indices}')

sorted_list = []
dct = {}
for i in range(len(orig)):
    sorted_list.append(orig[sort_indices[i]])
    # dct[sort_indices[i]] = i
    dct[i] = sort_indices[i]
print(f'sorted_list: {sorted_list}')
print(f'dct: {dct}')

orig: [20, 10, 15, 30, 25]
sort_indices: [1 2 0 4 3]
sorted_list: [10, 15, 20, 25, 30]
dct: {0: 1, 1: 2, 2: 0, 3: 4, 4: 3}


In [25]:
import numpy as np

adduct = [20, 10, 15, 30, 25]
print(f'adduct orig: {adduct}')

# adduct_sort
# i_orig = d[i_sort]

# adduct_sort
# adduct_sort = adduct.sort()  # sorts array/list in place
# print(f'adduct_sort: {adduct_sort}')
# print(f'adduct sorted: {adduct.sort()}')
# print(f'adduct orig: {adduct}')
# print(f'sorted adduct: {adduct}')

adduct = np.array([20, 10, 15, 30, 25])
# sorting
indices_sort = np.argsort(adduct)
print(f'indices of sorted array: {indices_sort}')
print(f'adduct sort: {[adduct[indices_sort[i]] for i in range(len(adduct))]}')

# i_orig = d[i_sort]
d = {}  # keys need to be indices of sorted array, values - corresponding indices of original array
print(f'first element of adduct: {adduct[indices_sort[0]]}')

for i_sort in range(len(indices_sort)):
    d[i_sort] = adduct[i_sort]
print(f'd using for loop: {d}')
keys = indices_sort
values = [i for i in range(len(adduct))]
print(f'keys: {keys}, values: {values}')
d = dict(zip(indices_sort, values))
# d = {keys[i]: values[i] for i in range(len(keys))}
# d = {indices_sort[i_orig]: adduct[indices_sort[i_orig]] for i_orig in range(len(adduct))}
print(f'dictionary of sorted indices: {d}')

adduct orig: [20, 10, 15, 30, 25]
indices of sorted array: [1 2 0 4 3]
adduct sort: [10, 15, 20, 25, 30]
first element of adduct: 10
d using for loop: {0: 20, 1: 10, 2: 15, 3: 30, 4: 25}
keys: [1 2 0 4 3], values: [0, 1, 2, 3, 4]
dictionary of sorted indices: {1: 0, 2: 1, 0: 2, 4: 3, 3: 4}


In [45]:
import numpy as np

adduct = [20, 10, 15, 30, 25]
print(f'adduct orig: {adduct}')

# adduct_sort
# i_orig = d[i_sort]

# adduct_sort
# adduct_sort = adduct.sort()  # sorts array/list in place
# print(f'adduct_sort: {adduct_sort}')
# print(f'adduct sorted: {adduct.sort()}')
# print(f'adduct orig: {adduct}')
# print(f'sorted adduct: {adduct}')

# adduct = np.array([20, 10, 15, 30, 25])
# sorting
indices_sort = np.argsort(adduct)
print(f'indices of sorted array: {indices_sort}')

#
# when we use adduct[indices_sort[i]], we effectively go from sorted array to original without using dictionary
#
print(f'adduct sort: {[adduct[indices_sort[i]] for i in range(len(adduct))]}')

# i_orig = d[i_sort]
d = {}  # keys need to be indices of sorted array, values - corresponding indices of original array
print(f'first element of adduct: {adduct[indices_sort[0]]}')

for i_sort in range(len(indices_sort)):
    d[i_sort] = adduct[i_sort]
print(f'd using for loop: {d}')

keys = indices_sort
values = [indices_sort[i] for i in range(len(adduct))]
print(f'keys: {keys}, values: {values}')
d = dict(zip(indices_sort, values))
# d = {keys[i]: values[i] for i in range(len(keys))}
d = {indices_sort[i_orig]: adduct[indices_sort[i_orig]] for i_orig in range(len(adduct))}
print(f'dictionary of sorted indices: {d}')

adduct orig: [20, 10, 15, 30, 25]
indices of sorted array: [1 2 0 4 3]
adduct sort: [10, 15, 20, 25, 30]
first element of adduct: 10
d using for loop: {0: 20, 1: 10, 2: 15, 3: 30, 4: 25}
keys: [1 2 0 4 3], values: [1, 2, 0, 4, 3]
dictionary of sorted indices: {1: 10, 2: 15, 0: 20, 4: 25, 3: 30}


In [51]:
import numpy as np

adduct = [20, 10, 15, 30, 25]
print(f'adduct orig: {adduct}')

indices_sort = np.argsort(adduct)
print(f'indices of sorted array: {indices_sort}')

#
# when we use adduct[indices_sort[i]], we effectively go from sorted array to original without using dictionary
#
adduct_sort = [adduct[indices_sort[i]] for i in range(len(adduct))]
print(f'adduct_sort: {adduct_sort}')

n = 3
print(f'Example: the adduct_sort[{n}] has value {adduct_sort[n]} has orig index {indices_sort[n]}')

n = 4
print(f'Example: the adduct_sort[{n}] with value {adduct_sort[n]} has orig index {indices_sort[n]}')

n = 2
print(f'Example: the adduct_sort[{n}] with value {adduct_sort[n]} has orig index {indices_sort[n]}')

adduct orig: [20, 10, 15, 30, 25]
indices of sorted array: [1 2 0 4 3]
adduct_sort: [10, 15, 20, 25, 30]
Example: the adduct_sort[3] has value 25 has orig index 4
Example: the adduct_sort[4] with value 30 has orig index 3
Example: the adduct_sort[2] with value 20 has orig index 0


In [55]:
adduct = [20, 10, 15, 30, 25]
print(f'adduct orig: {adduct}')

indices_sort = np.argsort(adduct)
print(f'indices of sorted array: {indices_sort}')

adduct_sort = [adduct[indices_sort[i]] for i in range(len(adduct))]
print(f'adduct_sort: {adduct_sort}')

for i in range(len(indices_sort)):
    print(f'i (new index): {i}, indices_sort (old index): {indices_sort[i]}')

adduct orig: [20, 10, 15, 30, 25]
indices of sorted array: [1 2 0 4 3]
adduct_sort: [10, 15, 20, 25, 30]
i (new index): 0, indices_sort (old index): 1
i (new index): 1, indices_sort (old index): 2
i (new index): 2, indices_sort (old index): 0
i (new index): 3, indices_sort (old index): 4
i (new index): 4, indices_sort (old index): 3


In [58]:
adduct = [20, 10, 15, 30, 25]
print(f'adduct orig: {adduct}')

indices_sort = np.argsort(adduct)
print(f'indices of sorted array: {indices_sort}')

adduct_sort = [adduct[indices_sort[i]] for i in range(len(adduct))]
print(f'adduct_sort: {adduct_sort}')

for i_sort in range(len(indices_sort)):
    print(f'i_sort (new index): {i_sort}, indices_sort (orig index): {indices_sort[i_sort]}')
    i_orig = indices_sort[i_sort]
    print(f'i_orig: {i_orig}')

adduct orig: [20, 10, 15, 30, 25]
indices of sorted array: [1 2 0 4 3]
adduct_sort: [10, 15, 20, 25, 30]
i_sort (new index): 0, indices_sort (orig index): 1
i_orig: 1
i_sort (new index): 1, indices_sort (orig index): 2
i_orig: 2
i_sort (new index): 2, indices_sort (orig index): 0
i_orig: 0
i_sort (new index): 3, indices_sort (orig index): 4
i_orig: 4
i_sort (new index): 4, indices_sort (orig index): 3
i_orig: 3


In [39]:
adduct = np.array([20, 10, 15, 30, 25])
print(f'adduct orig: {adduct}')
indices_orig = np.array([i for i in range(len(adduct))])
print(f'indices of orig array:   {indices_orig}')
# sorting
indices_sort = np.argsort(adduct)
print(f'indices of sorted array: {indices_sort}')

#
# when we use adduct[indices_sort[i]], we effectively go from sorted array to original without using dictionary
#
print(f'adduct sort: {[adduct[indices_sort[i]] for i in range(len(adduct))]}')
print()
# get list of orig indices from sorted indices
# orig array[sorted indices]
iadduct = [[i for i in range(len(indices_sort))]]
print(f'iadduct: {iadduct}')
print(f'orig array: {[adduct[iadduct] for iadduct in range(len(adduct))]}')

adduct orig: [20 10 15 30 25]
indices of orig array:   [0 1 2 3 4]
indices of sorted array: [1 2 0 4 3]
adduct sort: [10, 15, 20, 25, 30]

iadduct: [[0, 1, 2, 3, 4]]
orig array: [20, 10, 15, 30, 25]


In [31]:
def binary_search_sort(array_ind, array, low, high, x):
    if high >= low:
        midpt = (high + low) // 2
        
        if array[array_ind[midpt]] == x:
            # found: stop recursion
            print(f'array[array_ind[{midpt}]]: {array[array_ind[midpt]]}')
            return midpt

        # continue search in the half that contains element
        
        if array[array_ind[midpt]] > x:
            return binary_search_sort(array_ind, array, low, midpt - 1, x)
        else:
            return binary_search_sort(array_ind, array, midpt + 1, high, x)

    else:
        return -1

index = binary_search_sort(sort_indices, orig, 0, len(sort_indices) - 1, 15)
print(f'index = {index}, sort_indices[index] = {sort_indices[index]}, orig[sort_indices[index]] = {orig[sort_indices[index]]}')

array[array_ind[1]]: 15
index = 1, sort_indices[index] = 2, orig[sort_indices[index]] = 15


In [33]:
import numpy as np

orig = [20, 10, 15, 30, 25]
sort_indices = np.argsort(orig)
# print(f'orig: {orig}')
# print(f'sort_indices: {sort_indices}')

sorted_list = []
dct = {}
for i in range(len(orig)):
    sorted_list.append(orig[sort_indices[i]])
    # dct[sort_indices[i]] = i
    dct[i] = sort_indices[i]
# print(f'sorted_list: {sorted_list}')
# print(f'dct: {dct}')

def process_test_sorted(M, K_sorted, k_indices, N):
    """
    M is database of metabolites
    K_sorted is array of adducts sorted
    k_indices is list of indices returned by np.argsort
    N is our measured signals
    """
    list_pairs = []
    signal_meta_list = []
    
    # find imeta in M, iadduct in sorted array
    for isignal in range(len(N)):
        for imeta in range(len(M)):
            signal_meta_list.append(N[isignal] - M[imeta])
    
    # find adduct using binary search
    for element in signal_meta_list:
        adduct_index = binary_search_sort(k_indices, K_sorted, 0, len(K_sorted)-1, element)
            
    iadduct = [[i for i in range(len(indices_sort))]]

    list_pairs.append((imeta, iadduct))  # need to have index instead of K[k_indices[adduct_index]]
    return list_pairs

M = [0.000003, 0.000012, 0.000081, 0.000099, 0.000076, 0.000045, 0.000092, 0.000068, 0.000047]
K = [0.000002, 0.000045, -0.000063, -0.000009, -0.000050, 0.000048, 0.000070, -0.000037, 0.000056, -0.000008] 
N = [0.000079, 0.000094, 0.000084, 0.000052, 0.000064, 0.000055, 0.000070, 0.000079]
k_indices = np.argsort(K)
K_sorted = [K[k_indices[i]] for i in range(len(K))]
process_test_sorted(M, K_sorted, k_indices, N)

array[array_ind[0]]: -3.7e-05


[(8, 7e-05)]

In [44]:
K = np.array([0.000002, 0.000045, -0.000063, -0.000009, -0.000050, 0.000048, 0.000070, -0.000037, 0.000056, -0.000008])
print(f'orig array: {K}')
K_indices = np.argsort(K)
print(f'sorted array: {[K[K_indices[i]] for i in range(len(K))]}')
print(f'K_indices: {K_indices}')

keys = K_indices
values = [i for i in range(len(K))]
ind_d = dict(zip(keys, values))
print(f'index dictionary: {ind_d}')

orig array: [ 2.0e-06  4.5e-05 -6.3e-05 -9.0e-06 -5.0e-05  4.8e-05  7.0e-05 -3.7e-05
  5.6e-05 -8.0e-06]
sorted array: [-6.3e-05, -5e-05, -3.7e-05, -9e-06, -8e-06, 2e-06, 4.5e-05, 4.8e-05, 5.6e-05, 7e-05]
K_indices: [2 4 7 3 9 0 1 5 8 6]
index dictionary: {2: 0, 4: 1, 7: 2, 3: 3, 9: 4, 0: 5, 1: 6, 5: 7, 8: 8, 6: 9}


In [12]:
import numpy as np

orig = [20, 10, 15, 30, 25]
sort_indices = np.argsort(orig)
print(f'orig: {orig}')
print(f'sort_indices: {sort_indices}')

sorted_list = []
dct = {}
for i in range(len(orig)):
    sorted_list.append(orig[sort_indices[i]])
    # dct[sort_indices[i]] = i
    dct[i] = sort_indices[i]
print(f'sorted_list: {sorted_list}')
print(f'dct: {dct}')

orig: [20, 10, 15, 30, 25]
sort_indices: [1 2 0 4 3]
sorted_list: [10, 15, 20, 25, 30]
dct: {0: 1, 1: 2, 2: 0, 3: 4, 4: 3}


In [ ]:
# closure

def b_search_wrap(a_i, a, low, high, x):
    def b_search(a_i, low, high, x):
        pass
        # if a[a_i[midpt]] == x